# Use the Entso-e API to download the data programatically

Download the "day-ahead" data for Germany for: 
- Load forecast 
- Load realised
- Generation solar 
- Generation wind (onshore and offshore)

All units are in MW (Megawatts). 

"Day-ahead" data provides values for each quarter of an hour, i.e. 96 values per day.

In [ ]:
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup as bs

# Import dictionaries, security_token and day_ahead loading script

from Name_convention_dictionaries import DocumentTypeDict
from Name_convention_dictionaries import ProcessTypeDict
from Name_convention_dictionaries import AreaDict
from Name_convention_dictionaries import PsrTypeDict
from security_token import security_token
from Load_data_entsoe_day import Load_data_entsoe

# Reverse dictionaries to increase readibility

zone_dict = {v:k for k,v in AreaDict.items()}
document_type_dict = {v:k for k,v in DocumentTypeDict.items()}
process_type_dict = {v:k for k,v in ProcessTypeDict.items()}
generation_type_dict = {v:k for k,v in PsrTypeDict.items()}

In [ ]:
# support functions
def str_to_datetime(date_str):
    return datetime.datetime.strptime(date_str, '%Y%m%d%H%M')

def datetime_to_str(date):
    return datetime.datetime.strftime(date, "%Y%m%d%H%M")

## Specify region (country name or bidding zone according to Entso-e) and time horizon
Check Name_convention_dictionaries.py if unsure how the country or bidding zone is named by Entso-e.

In [ ]:
region = 'Germany'

# The Entso-e API only allows to download 380 days per call. So create a loop to download all data 
# Time-format: Year, Month, Day, Hour, Minute

num_of_years_to_download = 7
most_recent_date = '202010252345'
list_for_loop = [365]*num_of_years_to_download

## Download Load Data

In [ ]:
# Define furhter parameter to download the data
process_type = ['Day ahead', 'Realised'] # saved as a list to download forecast and realised respectively
document_type = 'System total load'

In [ ]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over forecast and realised load
    for process in process_type:
        
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process]\
              +'&outBiddingZone_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data
    
        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')

## Download generation data

This is a little bit more complex than the load data. Therefore it is necessary to first download the generation actuals (realised) and then the forecast. 

Download the actuals/realised values for generation:

In [ ]:
# Define furhter parameter to download the data
process_type = 'Realised' 
document_type = 'Actual generation per type'
generation_type = ['Solar', 'Wind Offshore', 'Wind Onshore']

In [ ]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over the generation types
    for generation in generation_type:
    
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process_type]\
              +'&psrType='+generation_type_dict[generation]\
              +'&in_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data

        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')

Download the forecasted generation for solar and wind

In [ ]:
# Define furhter parameter to download the data
process_type = 'Day ahead' 
document_type = 'Wind and solar forecast'
generation_type = ['Solar', 'Wind Offshore', 'Wind Onshore']

In [ ]:
last_date_to_download = most_recent_date

for year in list_for_loop:
    begin_time = datetime_to_str(str_to_datetime(last_date_to_download) - datetime.timedelta(days=year))
    end_time = last_date_to_download

    # loop over the generation types
    for generation in generation_type:
    
        # generate url for API call

        url = "https://transparency.entsoe.eu/api?securityToken="+security_token\
              +'&documentType='+document_type_dict[document_type]\
              +'&processType='+process_type_dict[process_type]\
              +'&psrType='+generation_type_dict[generation]\
              +'&in_Domain='+zone_dict[region]\
              +'&periodStart='+begin_time\
              +'&periodEnd='+end_time

        # load and save data

        try:
            Load_data_entsoe(url)
            last_date_to_download = begin_time
        except:
            print('\n No more data available \n')